In [1]:
import pandas as pd
import geopandas as gpd
from sklearn.decomposition import PCA
import numpy as np
import matplotlib.pyplot as plt

from sklearn.preprocessing import StandardScaler
from sklearn.metrics import r2_score
from sklearn.ensemble import RandomForestRegressor

from sklearn.model_selection import train_test_split
from sklearn.model_selection import PredefinedSplit
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import cross_val_score
from sklearn.multioutput import MultiOutputRegressor
import warnings
warnings.filterwarnings('ignore')

### Helper Functions

In [2]:
def loadData(file):
    data = pd.read_csv(file)
    print('Raw shape: ',data.shape)
    data['Date'] = pd.to_datetime(data.Date)
    print('Days: ',len(set(data.Date)))
    return data

In [3]:
def getTimeSeries(df,zones):
    df = df.loc[df['DOLocationID'].isin(zones)]
    table = pd.pivot_table(df, values='vehicle_count', index=['Date','Hour'],
                    columns=['DOLocationID'], aggfunc=np.sum, fill_value=0)
#     table.columns = [i[1] for i in table.columns]
    missing_columns = [i for i in zones if i not in table.columns]
    for col in missing_columns:
        table[col] = 0
    table = table[sorted(table.columns)]
    return table

In [4]:
def zscoreNormalizeSpatial(matrix):
    m = matrix.copy()
    for i in range(m.shape[0]):
        m[i, :] = (m[i, :] - m[i, :].mean()) / (m[i, :].std()+1e-10)
        
    return m

In [5]:
def standardize(matrix):
    m = matrix.copy()
    scaler = StandardScaler()
    scaler.fit(m)
    t = scaler.transform(m)
    return scaler, t
def inverse_standardize(matrix, scaler):
    t = matrix.copy()
    return scaler.inverse_transform(t)
def getPCAFeatures(matrix, n=10):
    pca = PCA(n_components=n)
    pca.fit(matrix)
    reducedMatrixPCA = pca.transform(matrix)
    reducedMatrixPCA.shape

    reducedDict = {str(i+1):reducedMatrixPCA[:,i] for i in range(reducedMatrixPCA.shape[1])}
    reducedDf = pd.DataFrame(reducedDict)
    #reducedDf.index = index
    return pca,reducedDf

In [6]:
def inverse_standardize(matrix, scaler):
    t = matrix.copy()
    return scaler.inverse_transform(t)

In [7]:
def getPCAFeatures(matrix, n=10):
    pca = PCA(n_components=n)
    pca.fit(matrix)
    reducedMatrixPCA = pca.transform(matrix)
    reducedMatrixPCA.shape

    reducedDict = {str(i+1):reducedMatrixPCA[:,i] for i in range(reducedMatrixPCA.shape[1])}
    reducedDf = pd.DataFrame(reducedDict)
    #reducedDf.index = index
    return pca,reducedDf

In [8]:
def inverse_pca(matrix,pca):
    m = matrix.copy()
    return pca.inverse_transform(m)

In [9]:
def addLag(dataset, maxlag, lagColumns):
    dataset_list = [dataset]

    for l in range(1, maxlag+1):
        df = dataset.shift(l)
        df = df[lagColumns]
        df.columns = [c+'_lag_'+str(l) for c in df.columns]
        dataset_list.append(df)

    dataset = pd.concat(dataset_list, axis=1).dropna()
    return dataset

#### train PCA from 2017

In [10]:
zone = gpd.read_file('../../Data/NYC Taxi Zones.geojson')
zones = zone['location_id'].unique()
zones = [int(i) for i in zones]

In [11]:
len(zones)

260

In [11]:
hub = 'LGA'
pca_comps = 6
dataDir = '/home/mingyi/Dropbox/UrbanTemporalNetworks/processedData/'
file = dataDir + hub + 'VehicleByHour2017fromHub.csv'

In [12]:
data2017 = loadData(file)
data2017 = getTimeSeries(data2017,zones)
data2017.head()

Raw shape:  (2251063, 4)
Days:  365


DOLocationID     1    2    3    4    5    6    7    8    9    10   ...  254  \
Date       Hour                                                    ...        
2017-01-01 0       0    0    0    0    0    0    2    0    0    0  ...    0   
           1       0    0    0    0    0    0    0    0    0    0  ...    0   
           2       0    0    0    0    0    0    0    0    0    0  ...    0   
           3       0    0    0    0    0    0    0    0    0    0  ...    0   
           4       0    0    0    0    0    0    0    0    0    0  ...    0   

DOLocationID     255  256  257  258  259  260  261  262  263  
Date       Hour                                               
2017-01-01 0       1    0    0    0    0    1    0    3    1  
           1       0    0    0    0    0    0    0    0    0  
           2       0    0    0    0    0    0    0    0    0  
           3       0    0    0    0    0    0    0    0    0  
           4       0    0    0    0    0    0    0    0    0  

[5 rows x 260 columns]

In [13]:
matrix2017 = data2017.values.astype(np.float64)
scaler, s_matrix2017 = standardize(matrix2017)
pca,pca_data2017 = getPCAFeatures(s_matrix2017,n=pca_comps)

In [14]:
r2_score(s_matrix2017,pca.inverse_transform(pca_data2017))

0.2898454380269946

#### Preparing Data

In [15]:
hub = 'LGA'
file = dataDir + hub + 'VehicleByHour2018fromHub.csv'

In [16]:
data = loadData(file)
data = getTimeSeries(data,zones)
data.shape

Raw shape:  (2260080, 4)
Days:  365


(8760, 260)

In [17]:
data

DOLocationID     1    2    3    4    5    6    7    8    9    10   ...  254  \
Date       Hour                                                    ...        
2018-01-01 0       0    0    0    0    0    0    1    0    1    0  ...    0   
           1       0    0    0    0    0    0    2    0    0    0  ...    0   
           2       0    0    0    0    0    0    0    0    0    0  ...    0   
           3       0    0    0    0    0    0    0    0    0    0  ...    0   
           4       1    0    0    0    0    0    0    0    0    0  ...    0   
...              ...  ...  ...  ...  ...  ...  ...  ...  ...  ...  ...  ...   
2018-12-31 19      0    0    1    0    0    0    8    0    1    0  ...    0   
           20      0    0    0    2    0    0   12    0    2    0  ...    1   
           21      0    0    1    1    0    0   18    0    1    0  ...    1   
           22      0    0    0    0    0    0   14    1    1    1  ...    1   
           23      1    0    1    2    0    0    7    0    1    1  ...    2   

DOLocationID     255  256  257  258  259  260  261  262  263  
Date       Hour                                               
2018-01-01 0       0    1    0    0    2    1    0    4    1  
           1       0    0    0    0    0    0    0    0    1  
           2       0    0    0    0    0    0    0    0    0  
           3       0    0    0    0    0    0    0    0    0  
           4       0    0    0    0    0    0    0    0    0  
...              ...  ...  ...  ...  ...  ...  ...  ...  ...  
2018-12-31 19      9    5    3    0    1    4    1    5    5  
           20      7    4    1    3    1    6    2   10   16  
           21      1    3    1    0    0    5    1    6    8  
           22      4    0    2    0    0    2    0    7    5  
           23      6    4    0    2    2    3    1    8    9  

[8760 rows x 260 columns]

In [18]:
matrix = data.values.astype(np.float64)
scaler, s_matrix = standardize(matrix)
pcaData = pd.DataFrame(pca.transform(s_matrix),columns=[str(i) for i in range(1,pca_comps+1)])

In [19]:
r2_score(s_matrix,pca.inverse_transform(pcaData))

0.3703096494768793

In [20]:
pcaData.index = data.index
pcaData = pcaData.reset_index()

In [21]:
externalDataDir = "/home/mingyi/Dropbox/UrbanTemporalNetworks/HongData/"
extFile = externalDataDir + hub.upper() + ".csv"

In [22]:
extDf = pd.read_csv(extFile)
print(extDf.shape)
extDf.head(2)

(8760, 46)


,date,arrival,fhv,yellow,vehicle,ifmon,iftue,ifwed,ifthu,iffri,...,maxtemp,mintemp,avgtemp,departure,hdd,cdd,participation,newsnow,snowdepth,ifSnow
0,18/1/1 0:00,3,89,67,156,1,0,0,0,0,...,19,8,13.5,-20.5,51,0,0.0,0.0,0,0
1,18/1/1 1:00,0,17,8,25,1,0,0,0,0,...,19,8,13.5,-20.5,51,0,0.0,0.0,0,0


In [23]:
extDf['date'] = pd.to_datetime(extDf['date'], yearfirst=True)
extDf.head(2)

,date,arrival,fhv,yellow,vehicle,ifmon,iftue,ifwed,ifthu,iffri,...,maxtemp,mintemp,avgtemp,departure,hdd,cdd,participation,newsnow,snowdepth,ifSnow
0,2018-01-01 00:00:00,3,89,67,156,1,0,0,0,0,...,19,8,13.5,-20.5,51,0,0.0,0.0,0,0
1,2018-01-01 01:00:00,0,17,8,25,1,0,0,0,0,...,19,8,13.5,-20.5,51,0,0.0,0.0,0,0


In [24]:
extDf['Hour'] = extDf['date'].dt.hour
extDf['Dow'] = extDf['date'].dt.dayofweek
extDf['Date'] = extDf['date'].dt.date

In [25]:
extDf.columns

Index(['date', 'arrival', 'fhv', 'yellow', 'vehicle', 'ifmon', 'iftue',
       'ifwed', 'ifthu', 'iffri', 'ifsat', 'ifsun', 'if0', 'if1', 'if2', 'if3',
       'if4', 'if5', 'if6', 'if7', 'if8', 'if9', 'if10', 'if11', 'if12',
       'if13', 'if14', 'if15', 'if16', 'if17', 'if18', 'if19', 'if20', 'if21',
       'if22', 'if23', 'maxtemp', 'mintemp', 'avgtemp', 'departure', 'hdd',
       'cdd', 'participation', 'newsnow', 'snowdepth', 'ifSnow', 'Hour', 'Dow',
       'Date'],
      dtype='object')

In [26]:
selected_columns = ['Date', 'Hour', 'Dow', 'arrival','maxtemp', 'mintemp', 'avgtemp', 'departure', 'hdd',
       'cdd', 'participation', 'newsnow', 'snowdepth', 'ifSnow']

In [27]:
extDf = extDf[selected_columns]

In [28]:
print(pcaData.shape)
print(extDf.shape)

(8760, 8)
(8760, 14)


In [29]:
pcaData['Date'] = pd.to_datetime(pcaData['Date'])
extDf['Date'] = pd.to_datetime(extDf['Date'])

In [30]:
pcaData

,Date,Hour,1,2,3,4,5,6
0,2018-01-01,0,-7.608241,2.277768,0.649247,0.436070,1.038062,-0.123052
1,2018-01-01,1,-11.259538,0.755065,-1.466972,-0.204779,-0.034775,0.129658
2,2018-01-01,2,-11.737270,0.691230,-1.826921,-0.640062,-0.073917,0.150433
3,2018-01-01,3,-11.765340,0.671444,-1.818517,-0.583366,-0.056369,-0.035902
4,2018-01-01,4,-11.456797,0.549760,-1.726690,0.042936,-0.198062,-0.312318
...,...,...,...,...,...,...,...,...
8755,2018-12-31,19,-0.150090,5.182543,1.835723,-1.598871,-0.471113,-0.909248
8756,2018-12-31,20,3.011138,5.585899,1.980323,-1.375512,-0.396871,1.789890
8757,2018-12-31,21,-1.869949,4.467263,2.275128,-0.663030,0.070394,0.965530
8758,2018-12-31,22,-0.595466,5.184895,3.617043,-0.969866,1.028351,0.631725


In [31]:
pd.merge(pcaData,extDf, on=['Date', 'Hour'], how='left')

,Date,Hour,1,2,3,4,5,6,Dow,arrival,maxtemp,mintemp,avgtemp,departure,hdd,cdd,participation,newsnow,snowdepth,ifSnow
0,2018-01-01,0,-7.608241,2.277768,0.649247,0.436070,1.038062,-0.123052,0,3,19,8,13.5,-20.5,51,0,0.00,0.0,0,0
1,2018-01-01,1,-11.259538,0.755065,-1.466972,-0.204779,-0.034775,0.129658,0,0,19,8,13.5,-20.5,51,0,0.00,0.0,0,0
2,2018-01-01,2,-11.737270,0.691230,-1.826921,-0.640062,-0.073917,0.150433,0,1,19,8,13.5,-20.5,51,0,0.00,0.0,0,0
3,2018-01-01,3,-11.765340,0.671444,-1.818517,-0.583366,-0.056369,-0.035902,0,0,19,8,13.5,-20.5,51,0,0.00,0.0,0,0
4,2018-01-01,4,-11.456797,0.549760,-1.726690,0.042936,-0.198062,-0.312318,0,0,19,8,13.5,-20.5,51,0,0.00,0.0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8755,2018-12-31,19,-0.150090,5.182543,1.835723,-1.598871,-0.471113,-0.909248,0,11,47,35,41.0,6.8,24,0,1.31,0.0,0,0
8756,2018-12-31,20,3.011138,5.585899,1.980323,-1.375512,-0.396871,1.789890,0,13,47,35,41.0,6.8,24,0,1.31,0.0,0,0
8757,2018-12-31,21,-1.869949,4.467263,2.275128,-0.663030,0.070394,0.965530,0,8,47,35,41.0,6.8,24,0,1.31,0.0,0,0
8758,2018-12-31,22,-0.595466,5.184895,3.617043,-0.969866,1.028351,0.631725,0,8,47,35,41.0,6.8,24,0,1.31,0.0,0,0


In [32]:
pcaData = pd.merge(pcaData,extDf, on=['Date', 'Hour'], how='left')
print(pcaData.shape)

(8760, 20)


In [33]:
pcaData.columns

Index(['Date', 'Hour', '1', '2', '3', '4', '5', '6', 'Dow', 'arrival',
       'maxtemp', 'mintemp', 'avgtemp', 'departure', 'hdd', 'cdd',
       'participation', 'newsnow', 'snowdepth', 'ifSnow'],
      dtype='object')

In [34]:
lagColumns = ['arrival'] + [str(i) for i in range(1,1+pca_comps)]

DateColumns = ['Date']

targetColumns = [str(i) for i in range(1,1+pca_comps)]

In [35]:
maxlag = 12

pcaData_lag = addLag(pcaData, maxlag, lagColumns)

pcaData_lag.shape

(8748, 104)

In [36]:
data.columns

Int64Index([  1,   2,   3,   4,   5,   6,   7,   8,   9,  10,
            ...
            254, 255, 256, 257, 258, 259, 260, 261, 262, 263],
           dtype='int64', name='DOLocationID', length=260)

In [37]:
pcaData_lag

,Date,Hour,1,2,3,4,5,6,Dow,arrival,...,4_lag_11,5_lag_11,6_lag_11,arrival_lag_12,1_lag_12,2_lag_12,3_lag_12,4_lag_12,5_lag_12,6_lag_12
12,2018-01-01,12,0.587249,2.425958,0.373209,0.257418,1.465838,-0.938493,0,14,...,-0.204779,-0.034775,0.129658,3.0,-7.608241,2.277768,0.649247,0.436070,1.038062,-0.123052
13,2018-01-01,13,-0.347753,2.722658,-1.361374,-0.734544,-0.293262,-0.417475,0,20,...,-0.640062,-0.073917,0.150433,0.0,-11.259538,0.755065,-1.466972,-0.204779,-0.034775,0.129658
14,2018-01-01,14,4.088727,3.851900,-1.927483,-0.012902,-1.192252,0.653483,0,25,...,-0.583366,-0.056369,-0.035902,1.0,-11.737270,0.691230,-1.826921,-0.640062,-0.073917,0.150433
15,2018-01-01,15,6.885754,4.284058,-2.126748,-2.148715,-0.670844,0.655847,0,20,...,0.042936,-0.198062,-0.312318,0.0,-11.765340,0.671444,-1.818517,-0.583366,-0.056369,-0.035902
16,2018-01-01,16,4.801734,3.950630,-4.095222,-0.786486,-0.725557,0.674121,0,24,...,-0.034388,-0.219773,0.113944,0.0,-11.456797,0.549760,-1.726690,0.042936,-0.198062,-0.312318
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8755,2018-12-31,19,-0.150090,5.182543,1.835723,-1.598871,-0.471113,-0.909248,0,11,...,-1.105700,-0.873693,0.817670,11.0,-8.859119,1.476983,-0.052346,-1.026466,0.741210,0.982009
8756,2018-12-31,20,3.011138,5.585899,1.980323,-1.375512,-0.396871,1.789890,0,13,...,-3.021247,-2.095873,-1.377630,22.0,-1.250566,2.790121,0.795951,-1.105700,-0.873693,0.817670
8757,2018-12-31,21,-1.869949,4.467263,2.275128,-0.663030,0.070394,0.965530,0,8,...,-2.869886,0.085247,-0.678427,23.0,2.979663,3.322754,4.256162,-3.021247,-2.095873,-1.377630
8758,2018-12-31,22,-0.595466,5.184895,3.617043,-0.969866,1.028351,0.631725,0,8,...,-3.119890,-0.120639,0.564033,28.0,9.214122,6.132787,3.000673,-2.869886,0.085247,-0.678427


In [38]:
CommR2List = []
EdgeR2List = []
residualDf_list = []
rawList = []
networkPrediction = pd.DataFrame()

for m in range(1,13):
    print()

    print("month: ",m)
    month_index  = pd.to_datetime(pcaData_lag.Date).dt.month == m

    dataset_train = pcaData_lag[~month_index]
    dataset_test = pcaData_lag[month_index]
    print("Train Size: ",dataset_train.shape)
    print("Test Size: ",dataset_test.shape)


    X_train = dataset_train.drop(targetColumns+DateColumns , axis = 1)
    X_test = dataset_test.drop(targetColumns+DateColumns , axis = 1)
    y_train = dataset_train[targetColumns]
    y_test = dataset_test[targetColumns]



    val_size = int(X_train.shape[0]*0.8)
    val_fold = list(-1*np.ones(X_train.shape[0]-val_size)) + list(np.zeros(val_size))
    ps = PredefinedSplit(val_fold)
    param_grid = [{
        "n_estimators": np.arange(10, 500, 50),
        "min_samples_split": np.arange(2, 50, 20),
        'min_samples_leaf': np.arange(2, 50, 20), 
        'max_features': ['sqrt'],
        'max_depth': np.arange(10, 50, 10),
    }]
    # fit_inverse_transform=True to make sure inverse transform available
    rf = RandomForestRegressor(random_state = 2019) 
    rf_grid_search = GridSearchCV(rf, param_grid, cv=ps, scoring='r2')
    rf_grid_search.fit(X_train, y_train)

    print("Train R2: ",rf_grid_search.best_estimator_.score(X_train,y_train))
    test_r2 = rf_grid_search.best_estimator_.score(X_test,y_test)
    print("Test R2: ",test_r2)


    pca_prediction = rf_grid_search.best_estimator_.predict(X_test)

    residual = y_test - pca_prediction
    residual_df = dataset_test[['Date','Hour']]
    residual_df = pd.concat([residual_df,pd.DataFrame(residual)], axis =1)

    network_prediction = inverse_pca(pca_prediction,pca)

    network_prediction = inverse_standardize(network_prediction, scaler)
    
    # relu to convert all prediction to positive
#     network_prediction = np.log(1+np.e**network_prediction)
    # round up negative values to 0
#     network_prediction = np.where(network_prediction<0,0,network_prediction)

    network_prediction_df = pd.DataFrame(network_prediction)
    network_prediction_df.columns = data.columns
    networkPrediction = pd.concat([networkPrediction,network_prediction_df])
    edgeMonthIndex = [False] * maxlag + list(month_index)
    edge_r2 = r2_score(data[edgeMonthIndex], network_prediction, multioutput='variance_weighted')
    print("Edge R2: ",edge_r2)


    CommR2List.append(test_r2)
    EdgeR2List.append(edge_r2)
    residualDf_list.append(residual_df)
#     rawList.append()


month:  1
Train Size:  (8016, 104)
Test Size:  (732, 104)
Train R2:  0.9669532493667599
Test R2:  0.83789946832628
Edge R2:  0.7430694049299414

month:  2
Train Size:  (8076, 104)
Test Size:  (672, 104)
Train R2:  0.9667896739667244
Test R2:  0.8773374341760939
Edge R2:  0.7716488226983421

month:  3
Train Size:  (8004, 104)
Test Size:  (744, 104)
Train R2:  0.9661202022649708
Test R2:  0.8981105603249971
Edge R2:  0.781737571496211

month:  4
Train Size:  (8028, 104)
Test Size:  (720, 104)
Train R2:  0.96676479321738
Test R2:  0.882268893599491
Edge R2:  0.7891855779615933

month:  5
Train Size:  (8004, 104)
Test Size:  (744, 104)
Train R2:  0.9664595479445297
Test R2:  0.8860652489745601
Edge R2:  0.79675341371411

month:  6
Train Size:  (8028, 104)
Test Size:  (720, 104)
Train R2:  0.9665957774564513
Test R2:  0.8828992422078046
Edge R2:  0.7839154211172387

month:  7
Train Size:  (8004, 104)
Test Size:  (744, 104)
Train R2:  0.966901451347947
Test R2:  0.8633618613123989
Edge R2: 

In [40]:
networkPrediction['Date'] = data.reset_index().iloc[12:]['Date'].values
networkPrediction['Hour'] = data.reset_index().iloc[12:]['Hour'].values
networkPrediction.to_csv('/home/mingyi/Dropbox/UrbanTemporalNetworks/prediction/learnFrom2017/from%sPCA%s.csv'%(hub,pca_comps),index=False)

In [39]:
print(np.mean(CommR2List))
print(np.mean(EdgeR2List))

0.8735618145466756
0.7690926181571953


In [41]:
networkPrediction[zones][networkPrediction[zones]<0] = 0
networkPrediction.to_csv('/home/mingyi/Dropbox/UrbanTemporalNetworks/prediction/learnFrom2017/from%sPCA%sRound.csv'%(hub,pca_comps),index=False)